

# transformers + deepspeed CLI



In [1]:
# Free colab seems to give different amount of general RAM to different users or even the same users at different times.

!free -h

               total        used        free      shared  buff/cache   available
Mem:            83Gi       877Mi        80Gi       5.0Mi       2.5Gi        81Gi
Swap:             0B          0B          0B


In [2]:
# check which nvidia drivers and cuda version is running

!nvidia-smi

Sat Feb 17 00:28:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
# Need ninja when build deepspeed from source code

!pip install ninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 2.5 MB/s eta 0:00:00


In [4]:
# 源代码安装 DeepSpeed
# 根据你的 GPU 实际情况（查看方法见前一页），设置参数 TORCH_CUDA_ARCH_LIST；
# 如果你需要使用 NVMe Offload，设置参数  DS_BUILD_UTILS=1；
# 如果你需要使用 CPU Offload 优化器参数，设置参数 DS_BUILD_CPU_ADAM=1；
!git clone https://github.com/microsoft/DeepSpeed/

Cloning into 'DeepSpeed'...
remote: Enumerating objects: 42803, done.
remote: Counting objects: 100% (42803/42803), done.
remote: Compressing objects: 100% (11340/11340), done.
remote: Total 42803 (delta 30886), reused 42629 (delta 30774), pack-reused 0
Receiving objects: 100% (42803/42803), 202.08 MiB | 47.34 MiB/s, done.
Resolving deltas: 100% (30886/30886), done.
Updating files: 100% (1500/1500), done.


In [5]:
%%bash


cd DeepSpeed
rm -rf build
TORCH_CUDA_ARCH_LIST="7.5" DS_BUILD_CPU_ADAM=1 DS_BUILD_UTILS=1 pip install -e . \
--global-option="build_ext" --global-option="-j8" --no-cache -v \
--disable-pip-version-check 2>&1 | tee build.log


Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
DEPRECATION: --build-option and --global-option are deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use --config-settings. Discussion can be found at https://github.com/pypa/pip/issues/11859
Obtaining file:///content/DeepSpeed
  Preparing metadata (setup.py): started
  Running command python setup.py egg_info
  [2024-02-17 00:29:23,217] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
  [2024-02-17 00:29:23,388] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
  DS_BUILD_OPS=0
   [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
   [WARNING]  async_io: please install the libaio-dev package with apt
   [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be

In [6]:
%%bash
git clone https://github.com/huggingface/transformers
cd transformers
pip install -e .
pip install -r examples/pytorch/translation/requirements.txt

Obtaining file:///content/transformers
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for transformers (pyproject.toml): started
  Building editable for transformers (pyproject.toml): finished with status 'done'
  Created wheel for transformers: filename=transformers-4.38.0.dev0-0.editable-py3-none-any.whl size=42166 sha256=1464ab87beb316e1f07e8e9ee0ce12d78bfc93c48df4302077b534c1456f0850
  Stored in directory: /tmp/pip-ephem-wheel-cache-6ihf3u9x/wheels/7c/35/80/e946b22a081210c6642e607ed65b2a5b9a4d9259695ee2caf5
Successfully b

Cloning into 'transformers'...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%bash

# DeepSpeed ZeRO-2 模式单 GPU 训练翻译模型（T5-Small）
cd transformers; export BS=16; rm -rf output_dir; \
PYTHONPATH=src USE_TF=0 CUDA_VISIBLE_DEVICES=0 deepspeed --num_gpus=1 examples/pytorch/translation/run_translation.py \
--deepspeed /content/drive/MyDrive/data/deepspeed/config/ds_config_zero2.json \
--model_name_or_path t5-small --per_device_train_batch_size 1 \
--output_dir output_dir --overwrite_output_dir --fp16 \
--do_train --max_train_samples 500 --num_train_epochs 1 \
--dataset_name wmt16 --dataset_config "ro-en" \
--source_lang en --target_lang ro


[2024-02-12 01:47:05,129] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-02-12 01:47:06,352] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
Detected CUDA_VISIBLE_DEVICES=0 but ignoring it because one or several of --include/--exclude/--num_gpus/--num_nodes cl args were used. If you want to use CUDA_VISIBLE_DEVICES don't pass any of these arguments to deepspeed.
[2024-02-12 01:47:06,353] [INFO] [runner.py:568:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None examples/pytorch/translation/run_translation.py --deepspeed /content/drive/MyDrive/data/deepspeed/config/ds_config_zero2.json --model_name_or_path t5-small --per_device_train_batch_size 1 --output_dir output_dir --overwrite_output_dir --fp16 --do_train --max_train_samples 500 --num_train_ep

Generating dataset wmt16 (/root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/27ea1f6483dca29955adc6a9e7d8a3556fbb1aea)
Dataset not on Hf google storage. Downloading and preparing it from source
hf://datasets/wmt16@7a79a3a94787a26ef461ae44b79cfd2ca2b27aef/ro-en/train/0000.parquet not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/downloads/e282454aa624184d75e04f8510187ad9e2b5b1823b80afc6262a115f3dfb4c37.incomplete
storing hf://datasets/wmt16@7a79a3a94787a26ef461ae44b79cfd2ca2b27aef/ro-en/train/0000.parquet in cache at /root/.cache/huggingface/datasets/downloads/e282454aa624184d75e04f8510187ad9e2b5b1823b80afc6262a115f3dfb4c37
creating metadata file for /root/.cache/huggingface/datasets/downloads/e282454aa624184d75e04f8510187ad9e2b5b1823b80afc6262a115f3dfb4c37
hf://datasets/wmt16@7a79a3a94787a26ef461ae44b79cfd2ca2b27aef/ro-en/validation/0000.parquet not found in cache or force_download set to True, downloading to /root/.cache/huggingface/da

In [13]:
%%bash

cd transformers; export BS=8; rm -rf output_dir; \
PYTHONPATH=src USE_TF=0 CUDA_VISIBLE_DEVICES=0 deepspeed --num_gpus=1 examples/pytorch/translation/run_translation.py \
--deepspeed /content/drive/MyDrive/data/deepspeed/config/ds_config_zero3.json \
--model_name_or_path t5-11b --output_dir output_dir --adam_eps 1e-06 \
--dataset_name wmt16  --dataset_config "ro-en" --source_lang en --target_lang ro \
--do_eval --do_predict --do_train --evaluation_strategy=steps \
--label_smoothing 0.1 --learning_rate 3e-5 --logging_first_step \
--logging_steps 1000 --max_source_length 128 --max_target_length 128 \
--num_train_epochs 1 --overwrite_output_dir --per_device_eval_batch_size $BS \
--per_device_train_batch_size $BS --predict_with_generate --eval_steps 25000 \
--sortish_sampler \
--val_max_target_length 128 --warmup_steps 5  \
--fp16



[2024-02-17 00:37:27,865] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-02-17 00:37:29,078] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
Detected CUDA_VISIBLE_DEVICES=0 but ignoring it because one or several of --include/--exclude/--num_gpus/--num_nodes cl args were used. If you want to use CUDA_VISIBLE_DEVICES don't pass any of these arguments to deepspeed.
[2024-02-17 00:37:29,078] [INFO] [runner.py:568:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None examples/pytorch/translation/run_translation.py --deepspeed /content/drive/MyDrive/data/deepspeed/config/ds_config_zero3.json --model_name_or_path t5-11b --output_dir output_dir --adam_eps 1e-06 --dataset_name wmt16 --dataset_config ro-en --source_lang en --target_lang ro --do_eval --do_pre

Generating dataset wmt16 (/root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/27ea1f6483dca29955adc6a9e7d8a3556fbb1aea)
Dataset not on Hf google storage. Downloading and preparing it from source
hf://datasets/wmt16@7a79a3a94787a26ef461ae44b79cfd2ca2b27aef/ro-en/train/0000.parquet not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/downloads/e282454aa624184d75e04f8510187ad9e2b5b1823b80afc6262a115f3dfb4c37.incomplete
storing hf://datasets/wmt16@7a79a3a94787a26ef461ae44b79cfd2ca2b27aef/ro-en/train/0000.parquet in cache at /root/.cache/huggingface/datasets/downloads/e282454aa624184d75e04f8510187ad9e2b5b1823b80afc6262a115f3dfb4c37
creating metadata file for /root/.cache/huggingface/datasets/downloads/e282454aa624184d75e04f8510187ad9e2b5b1823b80afc6262a115f3dfb4c37
hf://datasets/wmt16@7a79a3a94787a26ef461ae44b79cfd2ca2b27aef/ro-en/validation/0000.parquet not found in cache or force_download set to True, downloading to /root/.cache/huggingface/da

CalledProcessError: Command 'b'\ncd transformers; export BS=8; rm -rf output_dir; \\\nPYTHONPATH=src USE_TF=0 CUDA_VISIBLE_DEVICES=0 deepspeed --num_gpus=1 examples/pytorch/translation/run_translation.py \\\n--deepspeed /content/drive/MyDrive/data/deepspeed/config/ds_config_zero3.json \\\n--model_name_or_path t5-11b --output_dir output_dir --adam_eps 1e-06 \\\n--dataset_name wmt16  --dataset_config "ro-en" --source_lang en --target_lang ro \\\n--do_eval --do_predict --do_train --evaluation_strategy=steps \\\n--label_smoothing 0.1 --learning_rate 3e-5 --logging_first_step \\\n--logging_steps 1000 --max_source_length 128 --max_target_length 128 \\\n--num_train_epochs 1 --overwrite_output_dir --per_device_eval_batch_size $BS \\\n--per_device_train_batch_size $BS --predict_with_generate --eval_steps 25000 \\\n--sortish_sampler \\\n--val_max_target_length 128 --warmup_steps 5  \\\n--fp16\n\n'' returned non-zero exit status 247.